In [ ]:
# %% [markdown]
# Retrieval system using HOG descriptors + mAP@1 and mAP@5
# This notebook:
# 1. Builds (or loads) HOG descriptors for the database (BBDD)
# 2. Extracts HOG for the query images
# 3. Retrieves the most similar database images using cosine similarity
# 4. Evaluates using mAP@1 and mAP@5 based on gt_corresps.pkl

# %%
import os
import cv2
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from noise_filter import preprocess_image
import background_remover_w2 as background_remover
from image_split import split_images
from image_split_team5 import segment_multiple_paintings
import matplotlib.pyplot as plt

# --- paths (adjust to your structure) ---
QUERY_FOLDER = "../Data/Week4/qsd1_w4/"     # query images
DB_FOLDER    = "../Data/BBDD/"        # database (gallery) images
GT_PATH      = "../Data/Week4/qsd1_w4/gt_corresps.pkl"

# where to store precomputed DB descriptors
DESC_DB_PATH = "results/descriptors_db_hog.pkl"

os.makedirs("results", exist_ok=True)


In [ ]:
# %%
def compute_hog_descriptor(
    img_bgr,
    size=(128, 128),
    cell=(4, 4),
    block=(16, 16),
    block_stride=(8, 8),
    nbins=9,
):
    """
    Compute a HOG descriptor for a BGR image.
    We:
    1. convert to grayscale
    2. resize to a fixed size so all descriptors have the same length
    3. compute HOG
    4. L2-normalize the final vector
    """
    # 1) BGR -> Gray
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    # 2) fixed resize to make descriptor length stable
    gray = cv2.resize(gray, size)

    # 3) create HOG object with consistent parameters
    hog = cv2.HOGDescriptor(
        _winSize=size,
        _blockSize=block,
        _blockStride=block_stride,
        _cellSize=cell,
        _nbins=nbins,
    )
    desc = hog.compute(gray)  # shape (N, 1)
    desc = desc.flatten().astype(np.float32)

    # 4) L2 normalization (good for cosine and distance-based retrieval)
    norm = np.linalg.norm(desc)
    if norm > 0:
        desc = desc / norm

    return desc


def load_images_from_folder(folder):
    """
    Loads all .jpg/ images from a folder, sorted by filename.
    Returns:
        names: list of filenames
        imgs:  list of loaded cv2 images
    """
    names = sorted(
        [
            f
            for f in os.listdir(folder)
            if f.lower().endswith((".jpg"))
        ]
    )
    imgs = []
    for name in names:
        path = os.path.join(folder, name)
        img = cv2.imread(path)
        if img is None:
            print(f"⚠️ Could not read {path}")
            continue
        imgs.append(img)
    return names, imgs


def precision_at_k(retrieved_indices, gt_indices, k):
    """
    Compute precision@k for one query.

    retrieved_indices: list of database indices sorted by similarity (best first)
    gt_indices: list of correct database indices for that query (can be 1 or more)
    k: cutoff (1, 5, ...)

    precision@k = (# of retrieved items in top-k that are in GT) / k
    """
    retrieved_k = retrieved_indices[:k]
    hits = sum(1 for r in retrieved_k if r in gt_indices)
    return hits / k


def mean_average_precision_at_k(all_retrieved, all_gts, k):
    """
    Compute mean precision@k over all queries.
    (Many assignments call this mAP@k when GT is small.)
    """
    assert len(all_retrieved) == len(all_gts)
    precisions = []
    for retrieved, gts in zip(all_retrieved, all_gts):
        p = precision_at_k(retrieved, gts, k)
        precisions.append(p)
    return np.mean(precisions)

def validate_split(is_split, imgs, min_size_ratio=0.2):
    """
    Validate the result of segment_multiple_paintings.

    Parameters:
        is_split (bool): Whether the segmentation said it's split.
        imgs (list or np.ndarray): List [img1, img2] or a single image.
        min_size_ratio (float): Minimum ratio (relative to original image width/height)
                                for a crop to be considered valid.
    Returns:
        (bool, list or np.ndarray): same format as input but corrected.
    """
    # If no split, nothing to do
    if not is_split:
        return False, imgs

    # If there are not exactly 2 images, return as not split
    if not isinstance(imgs, (list, tuple)) or len(imgs) != 2:
        return False, imgs

    img1, img2 = imgs

    # Compute relative sizes compared to original (assume same height)
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    total_width = w1 + w2
    total_height = max(h1, h2)

    # Check each crop relative to the original combined width
    valid_left  = w1 / total_width  > min_size_ratio
    valid_right = w2 / total_width  > min_size_ratio
    valid_height1 = h1 / total_height > min_size_ratio
    valid_height2 = h2 / total_height > min_size_ratio

    # Only keep crops that are not too small
    valid_imgs = []
    if valid_left and valid_height1:
        valid_imgs.append(img1)
    if valid_right and valid_height2:
        valid_imgs.append(img2)

    # Decide if it's still a valid split
    if len(valid_imgs) == 2:
        return True, valid_imgs
    elif len(valid_imgs) == 1:
        # Only one valid crop left → treat as not split
        return False, valid_imgs[0]
    else:
        # none valid? probably invalid segmentation
        return False, imgs



In [ ]:
# %% 
# 1. Build or load database descriptors (HOG)

if os.path.exists(DESC_DB_PATH):
    print(f"✅ Loading DB descriptors from {DESC_DB_PATH}")
    with open(DESC_DB_PATH, "rb") as f:
        data = pickle.load(f)
        db_descs = data["desc_gt"]    # list of numpy arrays
        db_names = data["gt_names"]   # list of image filenames
else:
    print("🧠 Computing HOG descriptors for database images...")
    db_names, db_imgs = load_images_from_folder(DB_FOLDER)
    db_descs = []
    for img, name in zip(db_imgs, db_names):
        print("Processing image",name)
        d = compute_hog_descriptor(img)
        db_descs.append(d)

    # store only numpy arrays (serializable) → no cv2.KeyPoint here
    with open(DESC_DB_PATH, "wb") as f:
        pickle.dump({"desc_gt": db_descs, "gt_names": db_names}, f)
    print(f"✅ Saved DB descriptors to {DESC_DB_PATH}")


In [ ]:
# %%
# 2. Load query images and ground-truth correspondences

print("📥 Loading query images...")
q_names, q_imgs = load_images_from_folder(QUERY_FOLDER)

print("📥 Loading ground-truth correspondences...")
with open(GT_PATH, "rb") as f:
    gt_corresps = pickle.load(f)   # e.g. [[236], [107], [280, 285], ...]
print(f"→ {len(gt_corresps)} GT entries loaded")


In [ ]:
# %%
# 3. Compute HOG descriptors for queries

print("🧠 Computing HOG for queries...")
desc_query = []
for img, img_name in zip(q_imgs, q_names):
    print("Processing image",img_name)
    # Split possible multiple artworks
    img=preprocess_image(img)
    bool_split, splitted_imgs = segment_multiple_paintings(img)
    
    bool_val, imgs_val = validate_split(bool_split, splitted_imgs)

    if bool_val is True:
        splitted = imgs_val  # two artworks detected
        left_artwork, right_artwork = splitted

        iml, left_mask, left_output, _ = background_remover.remove_background_morphological_gradient(left_artwork)
        imr, right_mask, right_output, _ = background_remover.remove_background_morphological_gradient(right_artwork)

        # Crop each artwork to its mask bounding box (no black borders)
        left_cropped = background_remover.crop_to_mask_rectangle(left_artwork, left_mask)
        right_cropped = background_remover.crop_to_mask_rectangle(right_artwork, right_mask)

        # Extract descriptors
        #kps_left = detect_keypoints(left_cropped, METHOD)
        #kps_right = detect_keypoints(right_cropped, METHOD)
        
        desc_left  = compute_hog_descriptor(left_cropped)
        desc_right = compute_hog_descriptor(right_cropped)

        desc_query.append([desc_left, desc_right])
        
        """plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(left_cropped, cv2.COLOR_BGR2RGB))
        plt.title(f"Left Artwork: {img_name}")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(right_cropped, cv2.COLOR_BGR2RGB))
        plt.title(f"Right Artwork: {img_name}")
        plt.axis('off')
        
        plt.show()"""

    else:  # single artwork
        splitted = imgs_val
        im, mask, output, _ = background_remover.remove_background_morphological_gradient(img)

        # Crop to mask bounding box (remove black)
        cropped = background_remover.crop_to_mask_rectangle(img, mask)

        # Extract descriptor
        #kps = detect_keypoints(cropped, METHOD)
        desc = compute_hog_descriptor(cropped)
            
        desc_query.append([desc])  # keep structure consistent
        
        """plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        plt.title(f"{img_name}")
        plt.axis('off')
        plt.show()"""

# stack DB descriptors into a matrix so cosine_similarity is fast
db_mat = np.vstack(db_descs)   # shape (N_db, D)


In [ ]:
# --- 4. Compute mAP@1 and mAP@5 ---
def compute_map_at_k(desc_query, desc_gt, gt_corresps, k=5):
    query_aps = []

    for i, descs in enumerate(desc_query):
        # GT for this query
        q_gt = gt_corresps[i]
        if not isinstance(q_gt, list):
            q_gt = [q_gt]

        # Lengths
        n_crops = len(descs)
        n_gts = len(q_gt)

        # Make lengths match
        if n_crops > n_gts:
            # repeat last GT until same length
            q_gt = q_gt + [q_gt[-1]] * (n_crops - n_gts)
        elif n_crops < n_gts:
            # repeat last descriptor until same length
            descs = descs + [descs[-1]] * (n_gts - n_crops)

        crop_aps = []

        # Compute the AP for each image (descriptors taken with their gt labels)
        for desc, gt in zip(descs, q_gt):
            sims = cosine_similarity([desc], desc_gt)[0]
            ranked_indices = np.argsort(-sims)[:k]

            # AP for THIS (crop, gt) pair
            num_correct = 0
            precs = []

            for rank, idx in enumerate(ranked_indices, start=1):
                if idx == gt:   # exact match for this specific GT
                    num_correct += 1
                    precs.append(num_correct / rank)

            ap = np.sum(precs)
            crop_aps.append(ap)

        # average over pairs of this query
        query_aps.append(float(np.mean(crop_aps)) if crop_aps else 0.0)

    # final mAP
    return float(np.mean(query_aps)) if query_aps else 0.0


map1 = compute_map_at_k(desc_query, db_descs, gt_corresps, k=1)
map5 = compute_map_at_k(desc_query, db_descs, gt_corresps, k=5)

print(f"\n✅ mAP@1 = {map1:.4f}")
print(f"✅ mAP@5 = {map5:.4f}")
